In [11]:
# 在 Sz 的子空间进行构建 block Hamiltonian
# 只做一步 DMRG enlarge block, 观察如何在好量子数子空间内做 enlarge block
# ====================================================================
import numpy as np
from scipy.sparse import kron, identity, lil_matrix
from scipy.sparse.linalg import eigsh  # Lanczos routine from ARPACK
# We will use python's "namedtuple" to represent the Block and EnlargedBlock
# objects
from collections import namedtuple
#
Block = namedtuple("Block", ["length", "basis_size", "operator_dict", "basis_sector_array"])
EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict", "basis_sector_array"])
#
def is_valid_block(block):
    if len(block.basis_sector_array) != block.basis_size:
        return False
    for op in block.operator_dict.values():
        if op.shape[0] != block.basis_size or op.shape[1] != block.basis_size:
            return False
    return True
# This function should test the same exact things, so there is no need to
# repeat its definition.
is_valid_enlarged_block = is_valid_block
#
def H2(Sz1, Sp1, Sz2, Sp2):  # two-site part of H
    """Given the operators S^z and S^+ on two sites in different Hilbert spaces
    (e.g. two blocks), returns a Kronecker product representing the
    corresponding two-site term in the Hamiltonian that joins the two sites.
    """
    J = Jz = 1.
    return (
        (J / 2) * (kron(Sp1, Sp2.conjugate().transpose()) + kron(Sp1.conjugate().transpose(), Sp2)) +
        Jz * kron(Sz1, Sz2)
    )
#
def enlarge_block(block):
    """This function enlarges the provided Block by a single site, returning an
    EnlargedBlock.
    """
    mblock = block.basis_size
    o = block.operator_dict

    # Create the new operators for the enlarged block.  Our basis becomes a
    # Kronecker product of the Block basis and the single-site basis.  NOTE:
    # `kron` uses the tensor product convention making blocks of the second
    # array scaled by the first.  As such, we adopt this convention for
    # Kronecker products throughout the code.
    enlarged_operator_dict = {
        "H": kron(o["H"], identity(model_d)) + kron(identity(mblock), H1) + H2(o["conn_Sz"], o["conn_Sp"], Sz1, Sp1),
        "conn_Sz": kron(identity(mblock), Sz1),
        "conn_Sp": kron(identity(mblock), Sp1),
    }
    # kron(o["H"], identity(model_d)): 块哈密顿的希尔伯特空间扩大一个格点
    # kron(identity(mblock), H1)： 新加入格点的磁场项加入进来
    # H2(o["conn_Sz"], o["conn_Sp"], Sz1, Sp1)： 块边缘格点与新加格点的相互作用
    # kron(identity(mblock), Sz1)：边缘算符的希尔伯特空间扩大
    # kron(identity(mblock), Sp1)：边缘算符的希尔伯特空间扩大 


    # This array keeps track of which sector each element of the new basis is
    # in.  `np.add.outer()` creates a matrix that adds each element of the
    # first vector with each element of the second, which when flattened
    # contains the sector of each basis element in the above Kronecker product.
    enlarged_basis_sector_array = np.add.outer(block.basis_sector_array, single_site_sectors).flatten()
    # 做量子数的直和 （0.5,-0.5）直和 （0.5,-0.5） ==> (1, 0, 0, -1)

    return EnlargedBlock(length=(block.length + 1),
                        basis_size=(block.basis_size * model_d),
                        operator_dict=enlarged_operator_dict,
                        basis_sector_array=enlarged_basis_sector_array)
# 
def roatate_and_truncate(operator, transformation_matrix):
    """
    Transforms the operator to new (possibly truncated) basis given by "transformation_matrix"
    """
    return transformation_matrix.conjugate().transpose().dot(operator.dot(transformation_matrix))
#
def index_map(array):
    """
    Given an array, returns a dictionary that allows quick access to the 
    indices at which a given value occurs
    Example usage:
    >>> by_index = index_map([3, 5, 5, 7, 3])
    >>> by_index[3]
    [0, 4]
    >>> by_index[5]
    [1, 2]
    >>> by_index[7]
    [3]
    """
    d = {}
    for index, value in enumerate(array):
        d.setdefault(value, []).append(index)
    return d
#    
#
if __name__ == "__main__":
    #---------------------------------------------------------------#
    # Model-specific code for the Heisenberg XXZ chain
    model_d = 2  # single-site basis size
    single_site_sectors = np.array([0.5, -0.5]) # S^z sectors corresponding to the
                                                # single site basis elements 用于标记 Sz 量子数
    Sz1 = np.array([[0.5, 0], [0, -0.5]], dtype='d')  # single-site S^z
    Sp1 = np.array([[0, 1], [0, 0]], dtype='d')  # single-site S^+
    H1 = np.array([[0, 0], [0, 0]], dtype='d')  # single-site portion of H is zero
    #----------------------------------------------------------------#
    # 构建初始块儿
    # conn refers to the connection operator, that is, the operator on the edge of
    # the block, on the interior of the chain.  We need to be able to represent S^z
    # and S^+ on that site in the current basis in order to grow the chain.
    initial_block = Block(length=1, basis_size=model_d, operator_dict={
                        "H": H1,
                        "conn_Sz": Sz1,
                        "conn_Sp": Sp1,
                        }, basis_sector_array=single_site_sectors)
    # 初始块中多了一项，为basis_sector_array
    #----------------------------------------------------------------#
    # 只做 一步 DMRG 重整化
    #
    L = 4  # 链长为 4
    m = 20 # 保留态个数
    target_Sz = 0
    block = initial_block
    #
    while 2 * block.length < L:
        current_L = 2 * block.length + 2 # current superblock length
        current_target_Sz = int(target_Sz) * current_L // L
        print("current L =", current_L)
        # 执行一步 idmrg 
        # block, energy = single_dmrg_step(block, block, m=m, target_Sz=current_target_Sz)
        # print("E/L", energy / current_L)
        sys = block
        env = block
        print("---------System block information------------------")
        print("sys.basis_sector_array:", sys.basis_sector_array)
        print("sys_basis_by_sector:", index_map(sys.basis_sector_array))
        # Enlarge each block by a single site
        sys_enl = enlarge_block(sys)
        sys_enl_basis_by_sector = index_map(sys_enl.basis_sector_array)
        print("---------Enlarge block by a single site----------------")
        print("sys_enl.basis_sector_array:", sys_enl.basis_sector_array)
        print("sys_enl_basis_by_sector:", sys_enl_basis_by_sector)
        #
        if sys is env: # no need to recalculate a second time
            env_enl = sys_enl
            env_enl_basis_by_sector = sys_enl_basis_by_sector
        else:
            env_enl = enlarge_block(env)
            env_enl_basis_by_sector = index_map(env_enl.basis_sector_array)
        #
        assert is_valid_enlarged_block(sys_enl)
        assert is_valid_enlarged_block(env_enl)
        #
        # Construct the full superblock Hamiltonian
        m_sys_enl = sys_enl.basis_size
        m_env_enl = env_enl.basis_size
        print("------------basis_size of enlarged block-------------")
        print("sys_enl.basis_size:", m_sys_enl)
        print("env_enl.basis_size:", m_env_enl)
        #
        sys_enl_op = sys_enl.operator_dict
        env_enl_op = env_enl.operator_dict
        superblock_hamiltonian = kron(sys_enl_op["H"], identity(m_env_enl)) + \
                                kron(identity(m_sys_enl), env_enl_op["H"]) + \
                                H2(sys_enl_op["conn_Sz"], sys_enl_op["conn_Sp"],
                                    env_enl_op["conn_Sz"], env_enl_op["conn_Sp"])
        #
        # Build up a "restricted" basis of states in the target sector and
        # reconstruct the superblock Hamiltonian in that sector.
        sector_indices = {} # will contain indices of the new (restricted) basis
                            # for which the enlarged system is in a given sector
        restricted_basis_indices = []  # will contain indices of the old (full) basis, which we are mapping to
        for sys_enl_Sz, sys_enl_basis_states in sys_enl_basis_by_sector.items():
            sector_indices[sys_enl_Sz] = []
            env_enl_Sz = target_Sz - sys_enl_Sz
            print("-------loop in sys_enl_basis_by_sector--------")
            print("sector_indices[sys_enl_Sz]:", sector_indices)
            if env_enl_Sz in env_enl_basis_by_sector:
                for i in sys_enl_basis_states:
                    print("---------- loop in sys_enl_basis_states ------------")
                    i_offset = m_env_enl * i  # considers the tensor product structure of the superblock basis
                    print("i_offset:", i_offset)
                    for j in env_enl_basis_by_sector[env_enl_Sz]:
                        print("env_enl_basis_by_sector", env_enl_basis_by_sector)
                        print("env_enl_Sz", env_enl_Sz)
                        print("env_enl_basis_by_sector[env_enl_Sz]",env_enl_basis_by_sector[env_enl_Sz])
                        print("j", j)
                        current_index = len(restricted_basis_indices)  # about-to-be-added index of restricted_basis_indices
                        sector_indices[sys_enl_Sz].append(current_index)
                        restricted_basis_indices.append(i_offset + j)
        #
        print("sector_indices:\n", sector_indices)
        print("restricted_basis_indices:\n", restricted_basis_indices)
        break 

L = 4
---------System block information------------------
sys.basis_sector_array: [ 0.5 -0.5]
sys_basis_by_sector: {0.5: [0], -0.5: [1]}
---------Enlarge block by a single site----------------
sys_enl.basis_sector_array: [ 1.  0.  0. -1.]
sys_enl_basis_by_sector: {1.0: [0], 0.0: [1, 2], -1.0: [3]}
------------basis_size of enlarged block-------------
sys_enl.basis_size: 4
env_enl.basis_size: 4
-------loop in sys_enl_basis_by_sector--------
sector_indices[sys_enl_Sz]: {1.0: []}
---------- loop in sys_enl_basis_states ------------
i_offset: 0
env_enl_basis_by_sector {1.0: [0], 0.0: [1, 2], -1.0: [3]}
env_enl_Sz -1.0
env_enl_basis_by_sector[env_enl_Sz] [3]
j 3
current_index: 0
sector_indices: {1.0: [0]}
restricted_basis_indices: [3]
-------loop in sys_enl_basis_by_sector--------
sector_indices[sys_enl_Sz]: {1.0: [0], 0.0: []}
---------- loop in sys_enl_basis_states ------------
i_offset: 4
env_enl_basis_by_sector {1.0: [0], 0.0: [1, 2], -1.0: [3]}
env_enl_Sz 0.0
env_enl_basis_by_sector[en